## 功能
> 對 yolo 模型進行 fine-tune 的動作

In [ ]:
# 檢查虛擬環境
import sys
import numpy
import tensorflow as tf
import torch
import os

In [ ]:
from ultralytics import YOLO

# 讓系統只看見這一張 GPU（它變成 PyTorch/TensorFlow 的 index=0）
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# TensorFlow 限制 GPU 記憶體使用
max_memory = 24564
frac = 0.6
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.set_logical_device_configuration(gpus[0],
            [tf.config.LogicalDeviceConfiguration(memory_limit=max_memory*frac)]) # MB
        print(f"[TF] 設定 TensorFlow 使用最多 {max_memory*frac}MB GPU 記憶體")
    except RuntimeError as e:
        print("[TF] 設定失敗:", e)

# PyTorch 限制 GPU 記憶體使用比例
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # 因為 CUDA_VISIBLE_DEVICES 已讓這張卡成為 0
    torch.cuda.set_per_process_memory_fraction(frac, device=device)

device = "cuda"# if torch.cuda.is_available() else "cpu"
print("CUDA 是否可用:", torch.cuda.is_available(), ", device: ", device)
print("CUDA 數量:", torch.cuda.device_count())

In [ ]:
# 設置初始模型
model = YOLO("yolo11s.pt").to(device) 

# 開始 Fine-Tuning
model.train(data="Combined_Bottle_Waste(recheck)_stage2/data.yaml", 
            epochs=200,
            imgsz=640,
            device=device,
            mosaic = 0.5, # 降低馬賽克增強概率（預設 1.0），避免小目標被過度縮放後模糊。
            mixup = 0.1,
            multi_scale = True, # 啟用多尺度訓練（如 640~1280 隨機縮放），增強模型對尺寸變化的適應性。
            close_mosaic = 0, # 延後關閉馬賽克增強（預設 10），延長多尺度學習時間
            patience = 200
            ,resume = True 
           )